In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import os
import tensorflow as tf
from skimage import exposure

keras = tf.keras

In [2]:
model = keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=(224,224,3), weights='imagenet')

In [3]:
%%time
preprocessor = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=exposure.equalize_hist,
                                                           validation_split=0.2)
training_data = preprocessor.flow_from_directory("E:/Jupyter/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images/",
                                        target_size=(224, 224),
                                       subset='training')
val_data = preprocessor.flow_from_directory("E:/Jupyter/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images/",
                                        target_size=(224, 224),
                                       subset='validation')

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.
Wall time: 1.89 s


In [4]:
%%time
features = model(training_data[0][0])
print(features.shape)

E:\Anaconda3\envs\myenv\lib\site-packages\skimage\exposure\exposure.py:181: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  hist, bin_centers = histogram(image, nbins)


(32, 7, 7, 2048)
Wall time: 2.55 s


In [5]:
model.trainable = False
model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [6]:
global_average_layer = keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(features)
print(feature_batch_average.shape)

(32, 2048)


In [7]:
prediction_layer = keras.layers.Dense(43, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 43)


In [8]:
assembled_model = tf.keras.Sequential([
  model,
  global_average_layer,
  prediction_layer
])

In [9]:
assembled_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
loss0,accuracy0 = assembled_model.evaluate_generator(val_data, steps = 20, verbose=1)

20/20 [==============================] - 46s 2s/step - loss: 4.4602 - accuracy: 0.0125


In [11]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 4.46
initial accuracy: 0.01


In [12]:
%%time
history = assembled_model.fit_generator(training_data,
                    epochs=2,
                    validation_data=val_data)

Epoch 1/2
981/981 [==============================] - 10865s 11s/step - loss: 0.9156 - accuracy: 0.7341 - val_loss: 3.3847 - val_accuracy: 0.3118
Epoch 2/2
981/981 [==============================] - 10836s 11s/step - loss: 0.4299 - accuracy: 0.8674 - val_loss: 4.3849 - val_accuracy: 0.2830
Wall time: 6h 1min 41s


In [ ]:
assembled_model.summary()

In [1]:
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Accuracy')


NameError: name 'history' is not defined

Using Tensorflow Keras, transfer learning was implemented using a ResNetv2 50 pre-trained on Google's ImageNet image recognition dataset, sequentially layered with a global averaging layer and a prediction (output) layer. Output layer was one-hot encoded to 43 outputs, activation function was softmax, loss function was categorical cross-entropy, and optimization was Adam. Test data set was split into 80% test and 20% validation and shuffled. Initial loss and accuracy were 4.46 and 0.01, respectively. Training was done over two epochs because it takes around 3 hours for a single epoch to complete on dev platform. Final training loss and accuracy were 0.4299 and 0.8674 respectively.